# Setup

In [6]:
import os, yaml, sqlite3, hashlib
from datetime import datetime, timezone

from utils import extract_license_plate_number

# Load and extract configuration variables
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
arrivals_path = config['file_paths']['arrivals']
departures_path = config['file_paths']['departures']
db_path = os.path.join(config['database']['location'], config['database']['name']+'.db')

# Connect to the database (database is created if it does not already exist)
with sqlite3.connect(db_path) as db_conn:
    db_cur = db_conn.cursor()  # cursor to execute SQL statements on db_conn
    
    # Create tables if they do not already exist
    ## Arrivals
    # res = db_cur.execute('drop table arrivals')
    res = db_cur.execute(
    """create table if not exists arrivals(
        arrival_record_id string, license_plate_number string, arrival_ts_utc timestamp, arrival_comment string, arrival_created_at_utc timestamp
    )""")
    ## Departures
    # res = db_cur.execute('drop table departures')
    res = db_cur.execute(
    """create table if not exists departures(
        arrival_record_id string, license_plate_number string, departure_ts_utc timestamp, departure_comment string, departure_created_at_utc timestamp
    )""")
    
    # Create join view if it does not already exist
    # res = db_cur.execute('drop view records_view')
    res = db_cur.execute(
    """
    create view if not exists records_view as
    select a.*, b.departure_ts_utc, b.departure_comment, b.departure_created_at_utc
    from arrivals a
       left join departures b on a.arrival_record_id = b.arrival_record_id
           and a.license_plate_number = b.license_plate_number
           and a.arrival_ts_utc < b.departure_ts_utc
    """
    )

# Arrivals

In [15]:
new_arrivals = os.listdir(os.path.join(arrivals_path, 'staging'))
# If there are new arrivals in the staging folder, add them to the database
if len(new_arrivals) > 0:
    for arrival_image in new_arrivals:
        image_path = os.path.join(arrivals_path, 'staging', arrival_image)
        license_plate_number = extract_license_plate_number(image_path)
        arrival_ts_utc = datetime.strftime(datetime.fromtimestamp(os.path.getctime(image_path), timezone.utc), '%Y-%m-%d %H:%M:%S.%f')
        arrival_created_at_utc = datetime.strftime(datetime.utcnow(), '%Y-%m-%d %H:%M:%S.%f')
        arrival_comment = arrival_image
        arrival_record_id = hashlib.md5(f'{arrival_image};{arrival_ts_utc}'.encode('utf-8')).hexdigest()

        with sqlite3.connect(db_path) as db_conn:
            db_cur = db_conn.cursor()  # cursor to execute SQL statements on db_conn
            res = db_cur.execute(
            f"""
            insert into arrivals (arrival_record_id, license_plate_number, arrival_ts_utc, arrival_comment, arrival_created_at_utc)
            values('{arrival_record_id}', '{license_plate_number}', '{arrival_ts_utc}', '{arrival_comment}', '{arrival_created_at_utc}')
            """
            )
            res = db_cur.execute(f"select * from arrivals where arrival_record_id = '{arrival_record_id}'").fetchall()
            if len(res) > 0:
                print('Arrival Record Inserted - OK')
                print(res)
                os.rename(
                    os.path.join(arrivals_path, 'staging', arrival_image),
                    os.path.join(arrivals_path, 'production', arrival_image)
                )
            else:
                print('Arrival Record Inserted - FAIL')
                print(arrival_image)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Arrival Record Inserted - OK
[('e240792e33bfb393df39c7858e55bde7', 'BPNW-958', '2023-12-29 19:05:50.942569', '1997_Ontario_license_plate_BPNW♔958.png', '2023-12-29 19:16:29.774401')]


# Departures

In [ ]:
# If there are new departures in the staging folder, add them to the database
new_departures = os.listdir(os.path.join(departures_path, 'staging'))